#### Imports

In [1]:
#We are obtaining the bayesian networks from https://www.bnlearn.com/bnrepository
from copy import deepcopy
from pgmpy.readwrite import BIFReader
from pgmpy.inference import VariableElimination
import random
from bayesianNetwork import *
from datasetManipulation import *

##### Gum code

In [2]:
"""
Gum Code that might be needed later
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

def datasetFromBayesianNetwork(model, n):
    g = gum.BNDatabaseGenerator(model)
    g.setRandomVarOrder()
    g.drawSamples(n)
    return g.to_pandas()
    
How to get the posterior of a bayesian network
pred = gum.getPosterior(cancerBN,evs={"Pollution":'high',"Smoker":'True'},target='Cancer')
instance = gum.Instantiation(pred)
instance.setFirst()

for _ in range(pred.domainSize()):
    print(instance.todict(True))
    print(pred.get(instance))
    instance.inc()
 
    """

'\nGum Code that might be needed later\n\ndef datasetFromBayesianNetwork(model, n):\n    g = gum.BNDatabaseGenerator(model)\n    g.setRandomVarOrder()\n    g.drawSamples(n)\n    return g.to_pandas()\n    \nHow to get the posterior of a bayesian network\npred = gum.getPosterior(cancerBN,evs={"Pollution":\'high\',"Smoker":\'True\'},target=\'Cancer\')\ninstance = gum.Instantiation(pred)\ninstance.setFirst()\n\nfor _ in range(pred.domainSize()):\n    print(instance.todict(True))\n    print(pred.get(instance))\n    instance.inc()\n \n    '

### Create the network and the dataset

In [5]:
random.seed(10)

BNmodel = BIFReader("cancer.bif").get_model()
BNInference = VariableElimination(BNmodel)
variableToPredict = "Cancer"

# Create a BNDatabaseGenerator object from the model
dataFromBN = datasetFromBayesianNetwork(BNmodel, 2000)
encodingDict, encodedDataset = encodeCategoricalColumns(dataFromBN)
dtTreeClassifier = decisionTreeFromDataset(encodedDataset, variableToPredict , 2)

dtAsNetwork = obtainNetworkXTreeStructure(dtTreeClassifier, encodedDataset.columns)

rootNodes = [node for node in dtAsNetwork.nodes if isRoot(node, dtAsNetwork)] 
root = rootNodes[0]

  0%|          | 0/5 [00:00<?, ?it/s]

### Compute the mean prediction of the decision tree

In [12]:
positiveProbPerLeafNode = {node : dtTreeClassifier.tree_.value[node][0][0] for node in dtAsNetwork.nodes if isLeaf(node, dtAsNetwork)}
print(meanForDTinBN(positiveProbPerLeafNode, dtAsNetwork, root, BNInference, {}, encodingDict, variableToPredict, 'True'))
BNInference.query(variables=['Cancer']).values

0.98837


array([0.01163, 0.98837])

In [27]:
query = BNInference.query(evidence={'Pollution':'high', 'Smoker':'True', 'Cancer' : 'True', 'Dyspnoea' : 'True'}, joint=True, variables = ['Xray'] )
feature = 'Xray'

In [30]:
query.get_value(**{feature :'negative'})

0.1